<img src="https://github.com/LinkedEarth/Logos/blob/master/pyLiPD_logo1_transparent.png?raw=true">

# Editing LiPD Files

## Authors

[Deborah Khider](https://orcid.org/0000-0001-7501-8430)


## Preamble

Now that we have learned about the [`Dataset` class](L3_dataset_class.ipynb) and how to extract information from it, let's edit a LiPD file. 